In [1]:
import os
# Changing the working directory
os.chdir('../..')
os.getcwd()

'/home/javiermunoz/TFM_DSBD'

In [2]:
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

from tensorflow.python.util import deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False

import tensorflow as tf
gpu_devices = tf.config.experimental.list_physical_devices("GPU")
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rc("figure", figsize=(15, 5))

import keras
from keras.models import Sequential 
from keras.layers import Dense, LSTM, Dropout, Flatten

import os 
from src.config import LM_PER_VIDEO, BASE_OUTPUT, TOTAL_LANDMARKS
from sklearn.utils import shuffle
from src.utils import load_dict

In [3]:
X_tens = load_dict("./data/subset_10_lsa_64/pckl_files/tf/npy_db_x.pkl")
Y_enc = load_dict("./data/subset_10_lsa_64/pckl_files/tf/npy_db_y.pkl")
le_mapping = load_dict("./data/subset_10_lsa_64/pckl_files/tf/labels_map.pkl")

In [4]:
# Assign the dataset splits
X_train, X_val, X_test = X_tens['train'], X_tens['val'], X_tens['test']
y_train, y_val, y_test = Y_enc['train'], Y_enc['val'], Y_enc['test']
n_classes = len(le_mapping)

# shuffling the data
X_train, y_train = shuffle(X_train, y_train)
X_val, y_val = shuffle(X_val, y_val)
X_test, y_test = shuffle(X_test, y_test)

In [5]:
# Mediapipe normalizes data for us
X_train[0].all() < 1 

False

In [6]:
X_train.shape, X_val.shape, X_test.shape

((393, 20, 1662), (72, 20, 1662), (35, 20, 1662))

In [7]:
y_train.shape, y_val.shape, y_test.shape

((393,), (72,), (35,))

In [8]:
model = Sequential([
    LSTM(units=128, activation='relu', return_sequences=True, input_shape=(LM_PER_VIDEO, TOTAL_LANDMARKS)),
    Dropout(0.2),
    LSTM(units=256, activation='relu', return_sequences=True),
    Dropout(0.2),
    LSTM(units=128, activation='relu', return_sequences=True),
    Dropout(0.2),
    LSTM(units=64, activation='relu', return_sequences=False),
    Dropout(0.2),
    Dense(units=64, activation='relu'),
    Dense(units=n_classes, activation='softmax'),
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 20, 128)           916992    
                                                                 
 dropout (Dropout)           (None, 20, 128)           0         
                                                                 
 lstm_1 (LSTM)               (None, 20, 256)           394240    
                                                                 
 dropout_1 (Dropout)         (None, 20, 256)           0         
                                                                 
 lstm_2 (LSTM)               (None, 20, 128)           197120    
                                                                 
 dropout_2 (Dropout)         (None, 20, 128)           0         
                                                                 
 lstm_3 (LSTM)               (None, 64)                4

In [9]:
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [10]:
# CReate a customized callback so when val_acc > 94% it stops training
from keras.callbacks import EarlyStopping, ModelCheckpoint

class MyThresholdCallback(tf.keras.callbacks.Callback):
    def __init__(self, threshold):
        super(MyThresholdCallback, self).__init__()
        self.threshold = threshold

    def on_epoch_end(self, epoch, logs=None): 
        val_acc = logs["accuracy"]
        if val_acc >= self.threshold:
            self.model.stop_training = True

In [11]:
epochs = 500
bst_model_path = BASE_OUTPUT + "best_weights.h5"

callbacks = [
    MyThresholdCallback(threshold=0.94),
    #ModelCheckpoint(filepath = bst_model_path, save_best_only=True, save_weights_only=True, monitor="accuracy")
]

history = model.fit(
    X_train,
    y_train,
    validation_data=(X_val, y_val),
    epochs=epochs,
    callbacks=callbacks,
    batch_size=32
)

Epoch 1/500
13/13 [==============================] - 4s 81ms/step - loss: 2.2993 - accuracy: 0.1323 - val_loss: 2.2729 - val_accuracy: 0.1667
Epoch 2/500
13/13 [==============================] - 1s 55ms/step - loss: 2.2594 - accuracy: 0.1196 - val_loss: 2.0895 - val_accuracy: 0.2917
Epoch 3/500
13/13 [==============================] - 1s 54ms/step - loss: 2.1107 - accuracy: 0.1959 - val_loss: 2.0516 - val_accuracy: 0.2083
Epoch 4/500
13/13 [==============================] - 1s 54ms/step - loss: 1.9808 - accuracy: 0.2316 - val_loss: 1.8563 - val_accuracy: 0.1528
Epoch 5/500
13/13 [==============================] - 1s 54ms/step - loss: 1.9275 - accuracy: 0.2188 - val_loss: 1.7869 - val_accuracy: 0.2361
Epoch 6/500
13/13 [==============================] - 1s 55ms/step - loss: 1.8900 - accuracy: 0.2265 - val_loss: 1.7266 - val_accuracy: 0.1806
Epoch 7/500
13/13 [==============================] - 1s 54ms/step - loss: 1.7552 - accuracy: 0.2697 - val_loss: 1.7890 - val_accuracy: 0.2361
Epoch 

In [14]:
# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = model.evaluate(X_test, y_test, batch_size=32)
print("test loss, test acc:", results)

Evaluate on test data
2/2 [==============================] - 0s 10ms/step - loss: 12.0543 - accuracy: 0.3429
test loss, test acc: [12.054329872131348, 0.34285715222358704]


In [13]:
SPLITS = ['train', 'val', 'test']
DATA_PATH = f"./data/videos"

for sp in SPLITS:
    for gloss in os.listdir(os.path.join(DATA_PATH, sp)):
        frames = []
        for video_name in os.listdir(os.path.join(DATA_PATH, sp, gloss)):
            if "aug" in video_name:
                os.remove(os.path.join(DATA_PATH, sp, gloss, video_name))

FileNotFoundError: [Errno 2] No such file or directory: './data/videos/train'